Para o projeto, vamos utilizar a biblioteca PySUS criada por fccoelho.

Mais informações podem ser encontradas através dos links abaixo:

Github:  
https://github.com/AlertaDengue/PySUS

Documentação:  
https://pysus.readthedocs.io/en/latest/tutorials.html#working-with-sih-data

Para realizar o download, precisaremos das seguintes bibliotecas e métodos:

In [1]:
from pysus.online_data.SIH import download
from pysus.online_data import parquets_to_dataframe
import pandas as pd
import csv

Como o dataset é muito grande e só utilizaremos algumas colunas no projeto, vamos primeiro listar e escolher as colunas que serão utilizadas.

In [2]:
# Faz download de um pequeno subset dos dados (escolhi arbitrariamente jan/2015).
df_test = parquets_to_dataframe(download(states='AL', years=2015, months=1))

# Cria uma lista de listas com os nomes das colunas, visto que usaremos a função writerows.
lista = [[coluna] for coluna in df_test.columns.tolist()]

# Cria um csv com os nomes das colunas.
with open('nome_colunas.csv', 'w', newline='') as arquivo:
    csv_writer = csv.writer(arquivo)
    csv_writer.writerows(lista)

Após analizar as colunas, foram escolhidas as seguintes colunas:

In [3]:
colunas = [
    'UF_ZI',
    'ANO_CMPT',
    'MES_CMPT',
    'CEP',
    'MUNIC_RES',
    'SEXO',
    'IDADE',
    'CID_NOTIF',
    'VAL_TOT'
]

In [5]:
df_consolidado = pd.DataFrame()
erros = []

for ano in range(2014, 2023+1): # +1 visto que o método range() é não inclusivo
    for mes in range(1,12+1):
        print(f'Iniciando download: {mes}/{ano}...', end=' ')
        
        try:
            dados = download(states='AL', years=ano, months=mes)
            print(f'Download realizado com sucesso.')

            # Converte os dados baixados em um dataframe
            df = parquets_to_dataframe(dados)

            # Filtra apenas as colunas 
            df = df[colunas] 
            
            # Adiciona novos dados ao final
            df_consolidado = pd.concat([df_consolidado, df])
            
            print(f'Dataframe agora possuí {df_consolidado.shape[0]} linhas.\n')

        except:
            erros.append(f'{mes}/{ano}')
            print('Erro.')
        
print('Download concluído')
print(f'Erros: {len(erros)}')

Dataframe agora possuí 1183143 linhas.

Iniciando download: 12/2020... Download realizado com sucesso.
Dataframe agora possuí 1196872 linhas.

Iniciando download: 1/2021... Download realizado com sucesso.
Dataframe agora possuí 1209609 linhas.

Iniciando download: 2/2021... Download realizado com sucesso.
Dataframe agora possuí 1223606 linhas.

Iniciando download: 3/2021... Download realizado com sucesso.
Dataframe agora possuí 1237138 linhas.

Iniciando download: 4/2021... Download realizado com sucesso.
Dataframe agora possuí 1251502 linhas.

Iniciando download: 5/2021... Download realizado com sucesso.
Dataframe agora possuí 1265946 linhas.

Iniciando download: 6/2021... Download realizado com sucesso.
Dataframe agora possuí 1279585 linhas.

Iniciando download: 7/2021... Download realizado com sucesso.
Dataframe agora possuí 1294128 linhas.

Iniciando download: 8/2021... Download realizado com sucesso.
Dataframe agora possuí 1307371 linhas.

Iniciando download: 9/2021... Download re

In [6]:
erros

['7/2023', '8/2023', '9/2023', '10/2023', '11/2023', '12/2023']

In [8]:
df_consolidado.to_csv('SIH_SUS_AL-2014-2023.csv', sep=',', encoding='utf-8', index=False)
# df_consolidado.to_csv('SIH_SUS_AL-2014-2023.csv', sep=',', encoding='utf-8', index=False, compression='gzip')

Caso você tenha algum problema em realizar o download dessa forma, também é possível fazer o download de todos os arquivos dbc e converte-los.

In [24]:
import pysus.utilities.readdbc as dbc
df = dbc.read_dbc('dbc_files/RDAL2201.dbc')
df = df[colunas]
df.head()

,UF_ZI,ANO_CMPT,MES_CMPT,ESPEC,CGC_HOSP,N_AIH,IDENT,CEP,MUNIC_RES
0,270000,2022,01,02,12200259000912,2722101303878,1,57480000,270240
1,270000,2022,01,02,12200259000912,2722101303889,1,57480000,270240
2,270000,2022,01,01,,2722106423564,1,57048076,270430
3,270000,2022,01,01,,2722106424202,1,57060190,270430
4,270000,2022,01,01,,2722106424213,1,57545000,270330
